In [149]:
import qib
import numpy as np
from tenpy.models.lattice import Kagome
from tenpy.networks.site import SpinHalfSite

L = 12
lat = Kagome(Lx, Ly, [SpinHalfSite() for _ in range(3)], bc='periodic')

N = lat.N_sites
A = np.zeros((N, N), dtype=int)

#J = (1, 1, 1)
#h = (0, 0, 0)
J, g = (1, 3)


perms_1 = [[0, 4, 6, 10, 2, 5, 8, 11], [4, 6, 10, 0, 5, 8, 11, 2]]
perms_2 = [[0, 1, 2, 3, 6, 7, 8, 9], [1, 2, 3, 0, 7, 8, 9, 6]]
perms_3 = [[1, 4, 9, 11, 3, 5, 7, 10], 
           [4, 1, 11, 9, 5, 7, 10, 3]
           #[5, 7, 10, 3]
         ]

for perm in perms_1+perms_2+perms_3:
    for i in range(len(perm)//2):
        A[perm[2*i], perm[2*i+1]] = 1
        A[perm[2*i+1], perm[2*i]] = 1

lat_ = qib.lattice.CustomizedLattice((L,), A)
field = qib.field.Field(qib.field.ParticleType.QUBIT, lat_)
#hamil = qib.HeisenbergHamiltonian(field, J, h).as_matrix()
hamil = qib.IsingHamiltonian(field, J, 0, g).as_matrix()

In [150]:
import sys
import scipy

t = 0.125
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import construct_ising_local_term, reduce_list, X, I2, get_perms, construct_heisenberg_local_term
from ansatz_sparse import ansatz_sparse
import rqcopt as oc
from scipy.sparse.linalg import expm_multiply
from qiskit.quantum_info import random_statevector
from scipy.linalg import expm
from qiskit.quantum_info import state_fidelity

X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
Y = np.array([[0, -1j], [1j, 0]])
YZ = np.kron(Y, Z)
I2 = np.array([[1, 0], [0, 1]])

#hloc = construct_heisenberg_local_term(J, h, ndim=2)
hloc1 = J*np.kron(Z, Z)
hloc2 = g*(np.kron(X, I2)+np.kron(I2, X))/4
hloc = hloc1 + hloc2

state = np.array(random_statevector(2**L).data)
V2 = scipy.linalg.expm(-1j*t*hloc/2)
V1 = scipy.linalg.expm(-1j*t*hloc)


Vlist_start =  [YZ, V1, YZ, YZ, V1, YZ, YZ, V1, YZ]
control_layers = [0, 2, 3, 5, 6, 8]
Vlist_reduced = [V1, V1, V1]

perms_extended = [[perms_1[0]]] + [perms_1] + [[perms_1[0]], [perms_2[0]]] +\
                    [perms_2] + [[perms_2[0]], [perms_3[0]]] + [perms_3] + [[perms_3[0]]] 
perms_ext_reduced = [perms_1]  + [perms_2] + [perms_3]

print("Trotter error of the starting point: ", 1-state_fidelity(ansatz_sparse(Vlist_start, L, perms_extended, state), expm_multiply(
    1j * t * hamil, state)))

Trotter error of the starting point:  0.061383926016741586


In [ ]:
from optimize_sparse import optimize
import h5py

niter = 20
Vlist, f_iter, err_iter = optimize(L, hamil, t, Vlist_start, perms_extended, perms_reduced=perms_ext_reduced,
                                   control_layers=control_layers, rS=1, niter=niter)

with h5py.File(f"./results/kagome_TFIM_L{L}_t{t}_layers{len(Vlist)}.hdf5", "w") as f:
    f.create_dataset("Vlist", data=Vlist)
    f.create_dataset("f_iter", data=f_iter)
    f.create_dataset("err_iter", data=err_iter)
    f.attrs["L"] = L
    f.attrs["t"] = float(t)

Current error:  0.24720910922949463
Current error:  0.23334550936253654
Current error:  0.207132811558577
Current error:  0.16413107452051903
Current error:  0.13007017146366848
Current error:  0.12309633215368866
Current error:  0.11744204476926422
Current error:  0.11228101795777268
Current error:  0.10941022434990791
Current error:  0.10754790308532897
Current error:  0.10658435346099468
Current error:  0.10637925916866095


In [ ]:
state = random_statevector(2**L).data

print("Trotter error of the starting point: ", 1-state_fidelity(ansatz_sparse(Vlist_start, L, perms_extended, state), expm_multiply(
    1j * t * hamil, state)))
print("Trotter error of the optimized point: ", 1-state_fidelity(ansatz_sparse(Vlist, L, perms_extended, state), expm_multiply(
    1j * t * hamil, state)))